In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re

from collections import Counter

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer

from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import stopwords # Import the stop word list
from nltk.tokenize import WordPunctTokenizer

import math
import json
import csv
from collections import Counter

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import neural_network
from sklearn.model_selection import GridSearchCV
from sklearn import ensemble
from sklearn.decomposition import PCA
from sklearn.cluster import MeanShift, estimate_bandwidth
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics.pairwise import cosine_similarity

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Load up csv tweets
tweeters = ['elonmusk','NASA']

In [3]:
all_tweets = []
#for handle in handles:
#    with open('../data/{}_tweets.csv'.format(handle)) as f: 
#        all_tweets.append([pd.read_csv(line) for line in f])

elonmusk = pd.read_csv('../data/elonmusk_tweets.csv')

In [4]:
NASA = pd.read_csv('../data/NASA_tweets.csv')

In [5]:
elonmusk.columns

Index(['text', 'author'], dtype='object')

In [6]:
elonmusk[:17]

,text,author
0,@AKSINGH_S Yes,elonmusk
1,@KingLutron @marshall5912 @buckycovington @Mac...,elonmusk
2,@marshall5912 @buckycovington @MacTechGenius T...,elonmusk
3,@paulmacguyscott Upgrade to the latest S. Noth...,elonmusk
4,@Josh_Soto They are correct. It’s awesome. Sec...,elonmusk
5,"@JimmyRussellExe It does so constantly, but so...",elonmusk
6,@AntVenom @zacharyj98 Depends on when you plac...,elonmusk
7,"@Josh_Soto Three or four months, depending on ...",elonmusk
8,@zacharyj98 Good idea ...,elonmusk
9,@MacTechGenius That is true. Electric motors a...,elonmusk


In [8]:
print(stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [16]:
tok = WordPunctTokenizer()
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))

def clean_tweet(raw_tweet):
    # 1. Remove HTML
    soup = BeautifulSoup(raw_tweet)
    souped = soup.get_text()
    #
    # 2. Remove non-letters        
    #letters_only = re.sub("[^a-zA-Z]", " ", review_text)
    stripped = re.sub(combined_pat, '', souped)
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    letters_only = re.sub("[^a-zA-Z]", " ", clean)
    #
    # 3. Convert to lower case, split into individual words
    lower_case = letters_only.lower()                             
    #
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    #stops = set(stopwords.words("english"))                  
    # 
    # 5. Remove stop words
    #meaningful_words = [w for w in words if not w in stops]   
    #
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    #return( " ".join( meaningful_words ))   
    words = tok.tokenize(lower_case)
    return (" ".join(words)).strip()
    

In [17]:
len(elonmusk)

2748

In [18]:

for i in range(len(elonmusk)):
    #if( (i+1)%10000 == 0 ):
        #print "Tweets %d of %d has been processed" % ( i+1, nums[1] )                                                                    
    elonmusk['text'][i] = clean_tweet(elonmusk['text'][i])

In [19]:
elonmusk[:17]

,text,author
0,s yes,elonmusk
1,exactly,elonmusk
2,they can form a union whenever they want there...,elonmusk
3,upgrade to the latest s nothing will give you ...,elonmusk
4,soto they are correct it s awesome second best...,elonmusk
5,it does so constantly but so smoothly that you...,elonmusk
6,depends on when you placed your initial reserv...,elonmusk
7,soto three or four months depending on demand ...,elonmusk
8,good idea,elonmusk
9,that is true electric motors are far more reli...,elonmusk


In [23]:
elonmusk.iloc[10,0]

'tesla dual motor means there is a motor in front a motor in rear one is optimized for power one for range car'

In [24]:
for i in range(len(NASA)):
    #if( (i+1)%10000 == 0 ):
        #print "Tweets %d of %d has been processed" % ( i+1, nums[1] )                                                                    
    NASA['text'][i] = clean_tweet(NASA['text'][i])

In [41]:
# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
count_vect = CountVectorizer()

In [42]:
total_data =  pd.concat([elonmusk, NASA])

In [43]:
total_data.tail()

,text,author
2360,received a special welcome from as they were f...,NASA
2361,craft changed our understanding of the solar s...,NASA
2362,the golden record contains sounds and greeting...,NASA
2363,on sept our spacecraft will have been in space...,NASA
2364,launched years ago our advanced composition ex...,NASA


In [44]:
total_data['out'] = np.where(total_data['author']=='elonmusk',1,0)
X = total_data.drop(['author','out'])
y = total_data['out']

In [45]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [46]:
X_train.head()

,text,author,out
1589,and we start digging the tunnel tonight,elonmusk,1
682,apps assume the worst now i need to get back t...,elonmusk,1
2432,static fire test looks good pending data revie...,elonmusk,1
633,three people will leave planet earth tomorrow ...,NASA,0
479,say hello to my little friend,elonmusk,1


In [52]:
train_data_bow = count_vect.fit_transform(X_train['text'])
test_data_bow = count_vect.transform(X_test['text'])

In [55]:
tfid_vect = TfidfVectorizer()

In [56]:
train_data_tfid = tfid_vect.fit_transform(X_train['text'])

test_data_tfid = tfid_vect.transform(X_test['text'])

In [57]:
train_data_tfid.shape

(3425, 6619)

In [59]:
from sklearn.model_selection import cross_val_score
rfc = ensemble.RandomForestClassifier()
cross_val_score(rfc, train_data_bow, y_train, cv=10)

array([0.89504373, 0.89795918, 0.90087464, 0.90379009, 0.8483965 ,
       0.88921283, 0.92105263, 0.88888889, 0.89473684, 0.90322581])

In [62]:
lr = LogisticRegression()
cross_val_score(lr, train_data_bow, y_train, cv=10)

array([0.94752187, 0.95043732, 0.94460641, 0.95626822, 0.93002915,
       0.95043732, 0.95614035, 0.95614035, 0.94736842, 0.93841642])

In [66]:
np.mean([0.94752187, 0.95043732, 0.94460641, 0.95626822, 0.93002915,
       0.95043732, 0.95614035, 0.95614035, 0.94736842, 0.93841642])

0.947736583

In [63]:
svc = SVC()
cross_val_score(svc, train_data_bow, y_train, cv=10)

array([0.53644315, 0.53644315, 0.53644315, 0.53644315, 0.5393586 ,
       0.53644315, 0.54093567, 0.53508772, 0.53508772, 0.53958944])

In [64]:
lr_tfid = LogisticRegression()
cross_val_score(lr_tfid, train_data_tfid, y_train, cv=10)

array([0.93877551, 0.94460641, 0.92419825, 0.93294461, 0.9154519 ,
       0.94752187, 0.95321637, 0.94444444, 0.92982456, 0.92668622])

In [65]:
np.mean([0.93877551, 0.94460641, 0.92419825, 0.93294461, 0.9154519 ,
       0.94752187, 0.95321637, 0.94444444, 0.92982456, 0.92668622])

0.9357670140000002

In [ ]:
# Logistic Regression with Bag of Words is winning.
# It will be hard to beat this as it's already at almost 95% accuracy.

In [67]:
# We'll make 500 iterations, use 2-deep trees, and set our loss function.
params = {'n_estimators': 500,
          'max_depth': 2,
          'loss': 'deviance'}

# Initialize and fit the model.
clf = ensemble.GradientBoostingClassifier(**params)
out = cross_val_score(clf, train_data_tfid, y_train, cv=10)
print(np.mean(out))
out

0.9132804856116591


array([0.90962099, 0.90962099, 0.93586006, 0.93002915, 0.88921283,
       0.90962099, 0.93859649, 0.90643275, 0.9005848 , 0.90322581])

In [68]:
# We'll make 500 iterations, use 2-deep trees, and set our loss function.
params = {'n_estimators': 500,
          'max_depth': 2,
          'loss': 'deviance'}

# Initialize and fit the model.
clf = ensemble.GradientBoostingClassifier(**params)
out = cross_val_score(clf, train_data_bow, y_train, cv=10)
print(np.mean(out))
out

0.9182401687183226


array([0.9154519 , 0.93294461, 0.94169096, 0.92419825, 0.88921283,
       0.9212828 , 0.93567251, 0.91520468, 0.9005848 , 0.90615836])

In [69]:
# Logistic Regression with Bag of Words features is the winner